# adjusting dates in attribute table


In [1]:
import numpy as np
import pandas as pd
import os
import geopandas as gpd

ModuleNotFoundError: No module named 'geopandas'

In [ ]:
# load the shapefile in as lw,
datapath = "F:\LWM_withdatefix\LWM_2021.shp"
lwm = gpd.read_file(datapath)

lwm_fix = lwm.copy()
lwm_fix['date_new'] = None
dates_new = []

In [ ]:
## Loop through dates to fix formatting
for i in range(len(lwm)):
    #print(f'\r{i}', end='', flush=True) #print iteration

    date = lwm.loc[i,'Date'] # subset date
    date_len = len(date) # get length of date to help identify date format
    
    ### Replace / and - with underscore for consistent dates
    if '/' in date:
        date = date.replace("/", "_")
    elif '-' in date:
        date = date.replace("-", "_")

    date_str = date
 ##################### loop through different date formats #####################
    ##### Clean formats with YYYY or MM-DD-YYYY ######
    if date_len == 4: # if length is 4, contains only year
        date_str = date
    elif date_len == 10: # if length is 10, contains date as MM_DD_YYYY
        date_str = date
    
    ##### messier formats with missing leading zero or missing day: if len is between 4 and 10 ######
    elif date_len > 4:  ##& (date_len > 4): 
        date_split = date

        # first look at how many date entries there are (is it m-yyyy or m-d-yyyy)
        #if string contains three entries: m-d-yyyy or m-dd-yyyy
        if len(date_split.split('_')) == 3:
            month,day,year = date_split.split('_')
        #if string contains two entries there is no day: m-yyyy or mm-yyyy
        if len(date_split.split('_')) == 2:
            month, year = date_split.split('_')
            day = "01" # assume a day of 01?

        ### Add a leading zero to the month and date if they are only 1 character
        if len(month) == 1: # if month is missing leading zero
            month = "0%s" %month
        elif len(month) == 3:
            month = month[0:2]
        else:
            month = month
        if len(day) < 2: # if day is missing leading zero
            day = "0%s" %day
        # ## If day is actually the year (in format mm-yyyy)
        # if len(day) == 4:
        #     year = day
        #     month, day, year = month, '01', year

        date_str = "%s_%s_%s" %(month, day, year)
         #elif len(date) > 10: ### for 1 date dublicate mm-dd-yyyymm-dd-yyyy
    #    date_str = date[0:10]
    else: 
        print('unexpected date')
        print(date)
        
 #print(date, date_str)
    #df.iloc[i,:]['Date'].replace(date_str,inplace = True) #replace date with new datestring
    lwm.replace(lwm.loc[i,'Date'], date_str, regex=True, inplace = True)
    #print(lwm.iloc[i,:]['Date'])
    #print(lwm.iloc[7,:]['Date'])
    dates_new.append(date_str)

In [ ]:
lwm_fix['date_new'] = dates_new
lwm['Date'] = dates_new


#get your current working directory and save it as LWM_2021_datefix.shp 
output_path = "F:\LWM_withdatefix\LWM_datefix_test.shp"

lwm.to_file(output_path)
